In [6]:
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, path):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, 256)
        self.hidden_fc1 = nn.Linear(256, 128)
        self.hidden_fc2 = nn.Linear(128, 64)
        self.output_fc = nn.Linear(64, output_dim)

        self.network = nn.Sequential(
            OrderedDict(
                [
                    ("0", self.input_fc),
                    ("elu1", nn.ELU()),
                    ("2", self.hidden_fc1),
                    ("elu2", nn.ELU()),
                    ("4", self.hidden_fc2),
                    ("elu3", nn.ELU()),
                    ("mu", self.output_fc),
                ]
            )
        )
        self.load_network(path)

    def load_network(self, path):
        sd = torch.load(path)["model"]

        # clean the state dict and load it
        od2 = OrderedDict()
        for key in sd:
            key2 = str(key).replace("a2c_network.actor_mlp.", "")
            key2 = key2.replace("a2c_network.", "")
            if "a2c_network" in key2 or "value" in key2 or "sigma" in key2:
                continue
            else:
                print(key2)
                od2[key2] = sd[str(key)]
        # strictly load the state dict
        self.network.load_state_dict(od2, strict=True)
        print("Loaded MLP network from {}".format(path))

    def forward(self, x):
        return self.network(x)

In [9]:
filepath = "networks/acc_command_2_multiplier_disturbance.pth"


# load model into MLP network

model = MLP(17, 4, filepath)

zero_input = torch.zeros(1, 17)
with torch.no_grad():
    output = model(zero_input)
    print(output)

# # have the model as a set of numpy arrays that can be multiplied to one another

def get_model_weights(model):
    weights = []
    biases = []
    for name, param in model.named_parameters():
        print(name)
        if "weight" in name:
            weights.append(param.data.numpy().T)
        elif "bias" in name:
            biases.append(param.data.numpy())
    return weights, biases


weights, biases = get_model_weights(model)

# # make numpy array for each weight and bias and save them to a file as a text file that can be read by c++

for i, w in enumerate(weights):
    with open(f"networks/weight_{i}.txt", "w") as file:
        file.write(f"{w.shape[0]}\n{w.shape[1]}\n")
        print(w.shape)
        print(w)#, w.flatten())
        w = w.flatten()
        for item in w:
            file.write(f"{item}\n")

for i, b in enumerate(biases):
    with open(f"networks/bias_{i}.txt", "w") as file:
        # write shape in dim 0
        file.write(f"{b.shape[0]}\n")
        print(b.shape)
        b = b.flatten()
        for item in b:
            file.write(f"{item}\n")

print("Done")











0.weight
0.bias
2.weight
2.bias
4.weight
4.bias
mu.weight
mu.bias
Loaded MLP network from networks/acc_command_2_multiplier_disturbance.pth
tensor([[-1.8438e-04,  5.0467e-01,  9.3230e-02, -2.9466e-02]])
input_fc.weight
input_fc.bias
hidden_fc1.weight
hidden_fc1.bias
hidden_fc2.weight
hidden_fc2.bias
output_fc.weight
output_fc.bias
(17, 256)
[[-0.01250207  0.01663112  0.00866423 ...  0.00435116  0.08009952
  -0.21129264]
 [ 0.7748038   0.07587783 -0.05085535 ... -0.13258356 -0.00889337
   0.02463424]
 [-0.6327174   0.13020372  0.11861307 ...  0.01845818 -0.19888823
   0.01966745]
 ...
 [ 0.17717712 -0.0470652  -0.0296856  ...  0.15510847  0.11242943
  -0.20741701]
 [ 0.01679958  0.07810975 -0.08683077 ...  0.06576692 -0.02230075
   0.01167393]
 [ 0.06269515 -0.35107088  0.05043096 ...  0.14248988 -0.03144643
   0.07666203]]
(256, 128)
[[-0.25908226  0.02336972 -0.4415029  ... -0.3273674  -0.14353918
   0.25987777]
 [-0.23861471  0.15876147  0.06278118 ...  0.19135207  0.01001304
   0.05